In [ ]:
using Distributed
ifelse(nprocs()<2, addprocs(1), nothing);

In [ ]:
using LinearAlgebra
using Plots
# using Revise

In [ ]:
using LatticeQM
using LatticeQM.Operators: graphene, valleyoperator
using LatticeQM.Operators: addinterlayerbias!, setfilling!, gethaldane, addrashba!, addzeeman!, valleyoperator

# 2D lattice 

In [ ]:
plot(Geometries2D.honeycomb_AB(), 3; supercell=[0:5,0:5], size=(400,200))

Special field:

In [ ]:
b = 8*π/(√3 * 3) /(6*2π);

In [ ]:
latAA = Geometries2D.honeycomb_AA()
ks = kpath(latAA; num_points=800)
# valley0 = valleyoperator(lat, x->1; spinhalf=false)
valley0 = valleyoperator(latAA, x->1; spinhalf=false)
Operators.peierlsinplane!(valley0, latAA, [-b,0.0]);

hopsAA = graphene(latAA; mode=:nospin, tz=0.3, ℓinter=0.01, ℓintra=0.01)
Operators.addinterlayerbias!(hopsAA, latAA, -0.6)
Operators.peierlsinplane!(hopsAA, latAA, [-b,0.0]);

bands1 = getbands(hopsAA, ks, valley0)

p1 = plot(bands1, 1; ylabel="\$\\varepsilon/t\$", size=(330,260))
# plot(p1; colorbar=true, clims=(-1,1), size=(400,200))
nothing

In [ ]:
latAB = Geometries2D.honeycomb_AB()
ks = kpath(latAB; num_points=800)
valley0 = valleyoperator(latAB; spinhalf=false)
Operators.peierlsinplane!(valley0, latAB, [-b,0.0]);

hopsAB = graphene(latAB; mode=:nospin, tz=0.3, ℓinter=0.01, ℓintra=0.01)
Operators.addinterlayerbias!(hopsAB, latAB, -0.6)
Operators.peierlsinplane!(hopsAB, latAB, [-b,0.0])

bands2 = getbands(hopsAB, ks, valley0)

p2 = plot(bands2, 1; ylabel="\$\\varepsilon/t\$", size=(330,260))
# plot(p2; colorbar=true, clims=(-1,1), size=(400,200))
nothing

In [ ]:
plot!(p1, title="AA stacking")
plot!(p2, title="AB stacking")
plot(p1,p2, layout=(1,2), size=(600,200))

High-resolution 3d band plot:

In [ ]:
kgrid = LatticeQM.Utils.regulargrid(;nk=10^4)
kpoints = Structure.getB(lat) * kgrid
# kpoints = Structure.Lattices.foldBZ!(lat,kpoints)
bandmatrix = Spectrum.bandmatrix(hops,kgrid);

using DelimitedFiles
writedlm("kpoints.out", kpoints)
writedlm("bands.out", bandmatrix)

# 1D ribbon

In [ ]:
lat1 = Structure.reduceto1D(Geometries2D.honeycomb_AB(), [[1, 0] [0, 4]])
plot(lat1, 3; supercell=[0:10], size=(400,200))

In [ ]:
N=40;
tz=0.2;
V=0.2;
b = 8*π/(√3 * 3) /(6*2π);

In [ ]:
lat1   = Structure.reduceto1D(Geometries2D.honeycomb_AA(), [[1, 0] [0, N]])
ks = kpath(lat1; num_points=500) # ["Γ", "Γ2"]

pos = Operators.positionalong(lat1, Structure.basis(lat1,2))

hopsAAfinite = TightBinding.droplatdim(TightBinding.superlattice(hopsAA, [[1, 0] [0, N]]),2)
hopsAAfinite = DenseHops(hopsAAfinite)
# hops1 = graphene(lat1; mode=:nospin, tz=tz, ℓinter=0.01, ℓintra=0.01)
# Operators.addinterlayerbias!(hops1, lat1, V)
# Operators.peierlsinplane!(hops1, lat1, [b,0.0]);

bands1 = getbands(hopsAAfinite, ks, pos)
nothing

In [ ]:
p3 = plot(bands1, 1; ylabel="\$\\varepsilon/t\$", markersize=0.8, size=(500,300), opacity=0.85, clims=(0,maximum(pos)), colorbar=true, cquantile=1.0, csymmetric=false, colorbar_title="transverse position")
p5 = plot(bands1, 1; ylabel="\$\\varepsilon/t\$", ylim=(-0.5,0.5), markersize=0.8, size=(500,300), opacity=0.85, clims=(0,maximum(pos)), colorbar=true, cquantile=1.0, csymmetric=false, colorbar_title="transverse position")
nothing

In [ ]:
lat1   = Structure.reduceto1D(Geometries2D.honeycomb_AB(), [[1, 0] [0, N]])
ks = kpath(lat1; num_points=500) #, ["Γ", "Γ2"]

pos = Operators.positionalong(lat1, Structure.basis(lat1,2))

hops1 = graphene(lat1; mode=:nospin, tz=tz, ℓinter=0.01, ℓintra=0.01)
hops1 = DenseHops(hops1)
Operators.addinterlayerbias!(hops1, lat1, V)
Operators.peierlsinplane!(hops1, lat1, [b,0.0]);

bands2 = getbands(hops1, ks, pos)
nothing

In [ ]:
p4 = plot(bands2, 1; ylabel="\$\\varepsilon/t\$", markersize=0.8, size=(500,300), clims=(0,maximum(pos)), opacity=0.85, colorbar=true, cquantile=1.0, csymmetric=false, colorbar_title="transverse position")
p6 = plot(bands2, 1; ylabel="\$\\varepsilon/t\$", ylim=(-0.5,0.5), markersize=0.8, size=(500,300), clims=(0,maximum(pos)), opacity=0.85, colorbar=true, cquantile=1.0, csymmetric=false, colorbar_title="transverse position")
nothing

In [ ]:
plot!(p3, title="AA stacking")
plot!(p4, title="AB stacking")
plot(p3,p4,p5,p6, layout=(2,2), size=(700,350))

# Periodic superpotential 

In [ ]:
using LatticeQM.Utils: rotation2D
using LatticeQM.Structure.Lattices: getA, superlattice

In [ ]:
lat = Geometries2D.honeycomb_AB()
A = getA(lat)[1:2,1:2]

# Superlattice periodicities
n,m = 10,9
np,mp = round.(Int,inv(A)*rotation2D(2π/6)*A) * [n,m]

slat = Structure.Lattices.superlattice(lat, [[n,m] [np,mp]])
# Structure.foldPC!(slat);

L = norm(getA(slat)[:,1])
eL = getA(slat)[:,1]/norm(getA(slat)[:,1])
angle = acos(dot(eL,getA(lat)[:,1])/(norm(getA(lat)[:,1])))
println("Superlattice length: $L")
println("Superlattice angle: ", angle/π*180, "°")

In [ ]:
plot(slat, 3; supercell=[0:1,0:0], markersize=4, size=(600,400))

In [ ]:
G1 = Structure.getB(slat)[:,1]; G2 = Structure.getB(slat)[:,2]
L1 = Structure.getA(slat)[:,1]; L2 = Structure.getA(slat)[:,2]

fodd(x::AbstractVector; p0=zero(L1)) =  1/(3*√3) * (sin(2*π*dot(G1,x-p0)) + sin(2*π*dot(G2,x-p0)) - sin(2*π*dot(G1+G2,x-p0)))
feven(x::AbstractVector; p0=(L1+L2)/2) = (1/3 + 2/9 * (cos(2*π*dot(G1,x-p0)) + cos(2*π*dot(G2,x-p0)) + cos(2*π*dot(G1+G2,x-p0)))-0.5)
fodd2(x; sharpness=8, kwargs...) = tanh(sharpness*2*fodd(x; kwargs...))/2
feven2(x; sharpness=5, kwargs...) = tanh(sharpness*2*feven(x; kwargs...))/2

fodd3(x; kwargs...) = 0.333 * fodd2(3*x; kwargs...) + 0.333 * fodd2(2*x; kwargs...) + 0.333 * fodd2(x; kwargs...)
feven3(x; kwargs...) = 0.333 * feven2(3*x; kwargs...) + 0.333 * feven2(2*x; kwargs...) + 0.333 * feven(x; kwargs...)

In [ ]:
Z1 = feven.(eachcol(Structure.positions(slat)))
Z2 = fodd.(eachcol(Structure.positions(slat)))

p1 = plot(slat, Z1; supercell=[0:1,0:1], colorbar=true) #markercolor=:inferno,
p2 = plot(slat, Z2; supercell=[0:1,0:1], colorbar=true)

plot(p1,p2; markersize=1.5, size=(700,300))

In [ ]:
Z1 = feven3.(eachcol(Structure.positions(slat)))
Z2 = fodd3.(eachcol(Structure.positions(slat)))

p1 = plot(slat, Z1; supercell=[0:1,0:1], clims=(-0.5,0.5), colorbar=true) #, markercolor=:inferno
p2 = plot(slat, Z2; supercell=[0:1,0:1], clims=(-0.5,0.5), colorbar=true)

plot(p1,p2; markersize=1.5, size=(700,300))

In [ ]:
# valley = valleyoperator(slat, x->1; spinhalf=false);
valley = valleyoperator(slat; spinhalf=false);
nothing

backfolded bands:

In [ ]:
hops = graphene(slat; cellrange=2, format=:sparse, mode=:nospin, tz=0.0, ℓinter=0.05, ℓintra=0.05)
# Operators.addinterlayerbias!(hops, lat, V)
# Operators.peierlsinplane!(hops, lat, [-b,0.0]);

ks = kpath(slat, ["M2", "Γ", "K", "M", "K'", "Γ", "M"]; num_points=150)
bands2 = getbands(hops, ks, valley; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands2, 1; clims=(-1,1), colorbar_title="valley", colorbar=true, ylabel="\$\\varepsilon/t\$", size=(330,260))

adding a moiré potential:

In [ ]:
hops3 = graphene(slat; cellrange=2, format=:sparse, mode=:nospin, tz=0.0, ℓinter=0.05, ℓintra=0.05)
Operators.addchemicalpotential!(hops3, slat, x-> 0.3 * fodd3(x[1:3])) #0.3 * fodd3(x[1:3])
# Operators.addinterlayerbias!(hops3, slat, V)
# Operators.peierlsinplane!(hops3, slat, [-b,0.0]);

ks = kpath(slat, ["M2", "Γ", "K", "M", "K'", "Γ", "M"]; num_points=150)
bands3 = getbands(hops3, ks, valley; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands3; clims=(-1,1), colorbar=true, colorbar_title="valley", ylabel="\$\\varepsilon/t\$", size=(330,260))

adding in-plane B-field such that cones coincide:

In [ ]:
hops4 = graphene(slat; cellrange=2, format=:sparse, mode=:nospin, tz=0.0, ℓinter=0.15, ℓintra=0.15)
Operators.addchemicalpotential!(hops4, slat, x-> 0.3 * fodd3(x[1:3]))
Operators.addinterlayerbias!(hops4, slat, 0.005)
Operators.peierlsinplane!(hops4, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)]) #[cos(angle+2π/12),sin(angle+2π/12)]
valley2 = deepcopy(valley)
Operators.peierlsinplane!(valley2, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)])

ks = kpath(slat, ["M2", "Γ", "K", "M", "K'", "Γ", "M"]; num_points=150)
# ks = kpath(slat, ["γ", "γ1"]; num_points=150)
bands4 = getbands(hops4, ks, valley2; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands4; clims=(-1,1), colorbar=true, ylabel="\$\\varepsilon/t\$", size=(330,260))

Add interlayer bias:

In [ ]:
hops5 = graphene(slat; cellrange=2, format=:sparse, mode=:nospin, tz=0.0, ℓinter=0.2, ℓintra=0.2)
# Operators.addchemicalpotential!(hops5, slat, x-> 0.1 * fodd3(x[1:3]))
Operators.addinterlayerbias!(hops5, slat, x -> 0.1 + 0.1*fodd3(x[1:3]))
Operators.peierlsinplane!(hops5, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)]) #[cos(angle+2π/12),sin(angle+2π/12)]
valley2 = deepcopy(valley)
Operators.peierlsinplane!(valley2, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)])

ks = kpath(slat, ["M2", "Γ", "K", "M", "K'", "Γ", "M"]; num_points=150)
# ks = kpath(slat, ["γ", "γ1"]; num_points=150)
bands5 = getbands(hops5, ks, valley2; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands5; clims=(-1,1), colorbar=true, ylabel="\$\\varepsilon/t\$", size=(330,260))

Add moiré modulation to interlayer hopping

In [ ]:
using .Structure.Lattices: displaceZ!

slat2 = deepcopy(slat)
displaceZ!(slat2, p -> 0.5 * fodd3(p)+0.5 * feven2(p))
# Geometries2D.smoothdisplaceZ!(slat2,0.0,0.2; sharp=12);

In [ ]:
plot(slat2, 3; supercell=[0:0], clims=(-2.0,2.0), colorbar=true)

In [ ]:
hops5 = graphene(slat2; cellrange=2, format=:sparse, mode=:nospin, tz=0.07, ℓinter=0.25, ℓintra=0.25)
# Operators.addchemicalpotential!(hops5, slat, x-> 0.1 * feven2(x[1:3]))
Operators.addinterlayerbias!(hops5, slat, x -> 0.1 + 0.1*fodd3(x[1:3]))
Operators.peierlsinplane!(hops5, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)]) #[cos(angle+2π/12),sin(angle+2π/12)]
valley2 = deepcopy(valley)
Operators.peierlsinplane!(valley2, slat, sqrt(3)*(b/L)*[cos(angle),sin(angle)])

# ks = kpath(slat, ["μ2", "γ", "κ", "μ", "κ'", "γ", "μ"]; num_points=150)
ks = kpath(slat, ["Γ", "Γ1"]; num_points=150)
bands5 = getbands(hops5, ks, valley2; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands5; clims=(-1,1), colorbar=true, ylabel="\$\\varepsilon/t\$", size=(330,260))

# Moiré ribbon

In [ ]:
N=10
riblat   = Structure.reduceto1D(slat, [[1, 0] [0, N]])
ks = kpath(riblat; num_points=100); #, ["Γ", "Γ2"]

In [ ]:
plot(riblat, 3; supercell=[0:2], markersize=2, size=(600,400))

In [ ]:
hops4 = graphene(riblat; cellrange=1, format=:sparse, mode=:nospin, tz=0.1, ℓinter=0.16, ℓintra=0.16)
# Operators.addchemicalpotential!(hops4, slat, x-> 0.1 * feven(x[1:3]))
Operators.addinterlayerbias!(hops4, riblat, x -> 0.1 + 0.1*fodd3(x[1:3]))
Operators.peierlsinplane!(hops4, riblat, sqrt(3)*(b/L)*[cos(angle),sin(angle)]) #[cos(angle+2π/12),sin(angle+2π/12)]

nothing

In [ ]:
pos = Operators.positionalong(riblat, Structure.basis(riblat,2));

In [ ]:
bands4 = getbands(hops4, ks, pos; format=:sparse, num_bands=60)
nothing

In [ ]:
plot(bands4; colorbar=true, csymmetric=false, clims=(0,maximum(pos)), ylabel="\$\\varepsilon/t\$", size=(330,260))

In [ ]:
using ProgressMeter

prog = ProgressThresh(1e-5, "Minimizing:")
for val in exp10.(range(2, stop=-2, length=20))
    ProgressMeter.update!(prog, val)
    sleep(0.1)
end

In [ ]:
prog = ProgressUnknown("Titles read:")
for val in ["a" , "b", "c", "d"]
    ProgressMeter.next!(prog)
    if val == "c"
        ProgressMeter.finish!(prog)
        break
    end
    sleep(1)
end